In [46]:
import numpy as np
import pandas as pd 
import string
import random
import csv
import time
import math
import multiset 
import plotly.express as px
import json 
import pandas as pd 
import pickle
%load_ext memory_profiler

## Reading Data Files 

In [2]:
# The target file includes the original sequences that encodes the data. 
with open('targetBlawat et al.LDPC.txt') as f:
    reads = f.readlines()
for i in range(0,len(reads)):
    reads[i] = reads[i][0:len(reads[i])-1]
# reads 

In [3]:
np.size(reads)

9608

In [4]:
# The errors_shuffled file include the erroneous copies shuffled. 
with open('errors_shuffled.txt') as f:
    reads_err = f.readlines()
for i in range(0,len(reads_err)):
#     reads_err[i] = reads_err[i][0:len(reads_err[i])-1]
    reads_err[i] = reads_err[i].strip()
# reads_err

In [5]:
np.size(reads_err)

1161706

In [6]:
# evyat.txt include the erroneous copies clustered and labeled. 
with open('evyat.txt') as f:
    reads_cl = f.readlines()
for i in range(0,len(reads_cl)):
    reads_cl[i] = reads_cl[i].strip()
# reads_cl

# Functions


In [20]:
def hash_fun(x,a,w,l):
    ind = x.find(a)
    return x[ind:min(len(x),ind+w+l)]

def ind_st(st):
    #All 3-grams: AAA,AAG,AAC,AAT,AGA,AGC,...,TTA,TTG,TTC,TTT
    # A-0, G-1, C-2, T-3
    # index of CAT = Decimal representaion of (203)
    N_q = {"A":0,"C":1,"G":2,"T":3}
    dec = 0
    for i in range(0,len(st)):
        dec += N_q[st[i]]*(4**(len(st)-i-1))
    return dec 
    
    
def bin_sig(x,q):
    bs = [0]*(4**q) 
    for i in range(0,len(x)-q+1):
        st = x[i:i+q]
        bs[ind_st(st)] = 1
    bs_str = ''.join(str(e) for e in bs)
    return bs_str

def bin_sig_block(x,q,num_blocks):
    bs_str = ""
    block_len = math.floor(len(x)/num_blocks)
    for i in range(0,num_blocks-1):
        bs_str += bin_sig(x[i*block_len:(i+1)*block_len],q)
    bs_str += bin_sig(x[(num_blocks-1)*block_len:],q)
    return bs_str
        
        
def ham_dis(x,y):
    dis = 0
    for i in range(0,len(x)):
        if(x[i]!= y[i]):
            dis += 1
    return dis 

def rand_perm(w):
    return ''.join(random.choice('ACGT') for _ in range(w))

def merge(x,y,parent):
    parent[x]= min(parent[x],parent[y])
    parent[y]= min(parent[x],parent[y])
    return parent
    
def rep_find(inp,parent):
    temp = inp
    cnt = 0
    while(parent[temp]!=temp and cnt<10):
        cnt += 1
        temp=parent[temp]
    return temp 

def edit_dis(s1, s2):
    m=len(s1)+1
    n=len(s2)+1

    tbl = {}
    for i in range(m): tbl[i,0]=i
    for j in range(n): tbl[0,j]=j
    for i in range(1, m):
        for j in range(1, n):
            cost = 0 if s1[i-1] == s2[j-1] else 1
            tbl[i,j] = min(tbl[i, j-1]+1, tbl[i-1, j]+1, tbl[i-1, j-1]+cost)

    return tbl[i,j]

def find_in_arr(el,parent):
    cnt = 0
    for i in range(0,len(parent)):
        if parent[i] == el: 
            cnt += 1 
    return cnt 

    
def display_parent(parent):
    clstr = {}
    for i in range(0,len(parent)):
        clstr[i]=[]
    for i in range(0,len(parent)):
        clstr[rep_find(i,parent)].append(i)
    return clstr

def display_parent_ver2(parent,C_til_prev,parent_prev):
    clstr = C_til_prev
    for i in range(0,len(parent)):
        if(parent[i]!=parent_prev[i]):
#                 print(i,rep_find(i,parent_prev),clstr[rep_find(i,parent_prev)])
            clstr[rep_find(i,parent)].extend(clstr[i])
            clstr[rep_find(i,parent_prev)] = []
    return clstr

In [21]:
hash_fun("AGCTGAATGC","TGA",3,2)

'TGAAT'

In [22]:
hash_fun("AGCTGAATGC","ATG",3,2)

'ATGC'

In [23]:
hash_fun("AGCTGAATGC","CAT",3,2)

''

In [24]:
edit_dis("ACGTG","AGTC")

2

In [35]:
m = len(reads[0])
q =  3
num_blocks = 3
x = "AACCTCTAGACACACAACTATTAGTACCTATTATAAACACAACATAAGGTCAAGATAAGCAGAGA"
y = "AACACGAAATAACACACCACGAAGAGACACGACACAACACAACAGAGCACAACAGAGCACAACAC"
ham_dis(bin_sig_block(x,q,num_blocks),bin_sig_block(y,q,num_blocks))

49

In [36]:
x = "AACCTCTAGACACACAACTATTAGTACCTATTATAAACACAACATAAGGTCAAGATAAGCAGAGA"
y = "AACACGAAATAACACACCACGAAGAGACACGACACAACACAACAGAGCACAACAGAGCACAACAC"
edit_dis(x,y)

28

In [37]:
edit_dis("ACGTG","AGTC")

2

In [38]:
x = reads_err[1]
y = reads_err[215]
ham_dis(bin_sig_block(x,q,num_blocks),bin_sig_block(y,q,num_blocks))

18

In [39]:
edit_dis(x,y)

4

In [44]:
# Precompute all the binary signatures
q = 3
num_blocks = 3
bin_sig_arr = []
for i in range(0,len(reads_err)):
    bin_sig_arr.append(bin_sig_block(reads_err[i],q,num_blocks))


In [89]:
with open('bin_sig_arr_sample.pickle', 'wb') as f:
    pickle.dump(bin_sig_arr,f)

In [73]:
bin_sig_arr[0]

'010111000000000100011010100000001000000000000000100001000000110100001010000001000101001001110000000000100100100101000100000010000100001010011110000000000010000000100000101010001001001000010000'

# Clustering Algorithm

In [68]:
def rep_in_C(read,C_reps):
    lower = 0;
    upper = len(C_reps) - 1;
    while (lower <= upper):
        mid = lower + int((upper - lower) / 2)
#         print(upper,mid)
        res = -1
        if (read == (C_reps[mid][0])):
            return C_reps[mid][1]
        if (read > (C_reps[mid][0])):
            lower = mid + 1;
        else:
            upper = mid - 1;
    return -1;

def comp_clstrs(alg_clstr,org_clstr,gamma):
    num_exist = 0
    if(len(alg_clstr)>len(org_clstr)):
#         print(alg_clstr)
        return 0 
    else:
        for i in range(0,len(alg_clstr)):
            flg_exist = 0
            for j in range(0,len(org_clstr)):
                if(reads_err[alg_clstr[i]] == org_clstr[j]):
                    flg_exist = 1
                    num_exist +=1
                    break
            if(flg_exist == 0):
                return 0
        if(num_exist < gamma*len(org_clstr)):
            return 0
            
        return 1 
    
def calc_acrcy(clustering,C_dict,C_reps,gamma):
#     clustering = display_parent(parent)
    acrcy = 0
    for i in range(0,len(clustering)):
        if(len(clustering[i])>=1):
            acrcy = acrcy + comp_clstrs(clustering[i],C_dict[rep_in_C(reads_err[clustering[i][0]],C_reps)],gamma)
    return acrcy

def min_max(val1,val2):
    min_val = min(val1,val2)
    max_val = max(val1,val2)
    return min_val,max_val
    

In [69]:
# (Read, Cluster rep of the cluster to which the read belongs to)
C_reps = []
C_dict = {}
rep = reads_cl[0]
for i in range(1,len(reads_cl)):
    if reads_cl[i] != "":
        if reads_cl[i][0] == "*":
            if(len(C_reps)>0):
                C_dict[rep].pop()
                C_reps.pop()
            rep=reads_cl[i-1]
            C_dict[rep]=[]
        else:
#             print(C_dict)
            C_dict[rep].append(reads_cl[i])
            C_reps.append((reads_cl[i],rep))
C_reps.sort(key=lambda x:x[0])

In [70]:
C_reps[0:5]

[('AAAAAAACCGGAATATCAATCGAGCGAATGGGATACTACATTAAGGGATATCACTTGATCACAAG',
  'AACGAAACCGGAATATCAATCGAGCGAATGGGATACTACATTAAGGGATATCACTTGATCACAAG'),
 ('AAAAAAAGGTGAACTCGAAGACGTTAACGATATGACCCGTCAATTAAATGACGTATACACTACT',
  'AACTAAAGGTGAACTCGAAGATCGTTAACGATATGACCCGTCAATTAAATGACGTATACACTACT'),
 ('AAAAACACACTCAACGATACCTCTAACGGATAATCCGGTCACTAACATAAGAAGCACGCAAT',
  'AACTACAAACACACTCAACGATACTCTAACGGATAATCCGGTCACTAACATAAGAAGCACGCAAT'),
 ('AAAAACACCAATCAGATAAGAGTATAAGAGATAAGAACATCTACGAATCCACGCAATATGACG',
  'AACCAAACACCAATCAGATAAGAGTATAAGAGATAAGAACATCTACGAATCCACGCAATATGACG'),
 ('AAAAACACTTAGAAACATCAAGAGCAGCATATCTACCTCGAGTCCGTCTGCACACAACGTAGC',
  'AACTAAACACTTAGAAACATCAAGAGCAGCATATCAACCTCGAGTCCGTCTGCACACAACGTAGC')]

In [71]:
C_dict[list(C_dict.keys())[0]][0:5]

['AATCTCTAGACACACAACTATTAGTACCTATTATAAACACAACATAAGGTCAAGATAAGCAGAGA',
 'AACCTCTAGACACACAACTATTAGTACCTATTATAAACACAACATAAGGTCAAGATAAGCAGAGA',
 'AACCTCTAGACACACAACTATTAGTACCTATTACAAACACAACATAAGGTCAAGATAAGCAGAGA',
 'AACCTCTAGACACACAACTATTAGTACCTATTATAAACACAACATAAGGTCAAGATAAGCAGAGA',
 'AACCTCTAGACAACACAACTATTAGTACCTATTATAAACACAACATAAGGTCAAGATAAGCAGAGA']

In [66]:
dist_arr =[]
for i in range(1,len(reads_err)):
    dist_arr.append(ham_dis(bin_sig_arr[i],bin_sig_arr[0]))
dist_arr.sort()
# dist_arr[0:200]

In [67]:
for i in range(0,1000):
    if(dist_arr[i+1]-dist_arr[i]>10):
#         print(dist_arr[i])
        break
th_low = dist_arr[i] + 5
th_high = min(dist_arr[i+1] - 5, th_low+10)
print(th_high,th_low)

33 25


In [75]:
# r = 4pm 
r = 10

In [76]:
reads_err_ind  = [0]*(len(reads_err))
parent = [0]*(len(reads_err))
for i in range(0,len(reads_err)):
    reads_err_ind[i] = (i,reads_err[i]) 
    parent[i] = i

C_til = display_parent(parent)
acrcy_dict1 = {}
acrcy_dict2 = {}
acrcy_dict3 = {}
acrcy_dict4 = {}
acrcy_dict5 = {}
acrcy_dict6 = {}

time_acrcy_dict = {}
time_itr_dict = {}


# DESIRED_ACRCY = 0.99
# gamma = 0.9
card_C = len(reads)
local_comm = 220

# th_high = 30
# th_low  = 24
# r = 10
w = math.ceil(math.log(len(reads_err[0]),4))
l = math.ceil(math.log(len(reads_err),4))

lcl_step = 0


for lcl_step in range(0,local_comm):
# while(acrcy < DESIRED_ACRCY):   
    time_start = time.time()

    hash_select = [0]*(len(reads_err))    
    for i in range(0,len(C_til)):
        if(len(C_til[i])!=0):    
            hash_select[random.choice(C_til[i])]=1
            
    a = rand_perm(w)
    hash_C_til = [0]*(len(reads_err))
    for i in range(0,len(reads_err_ind)):
        if(hash_select[i]==0):
            hash_C_til[i] = (i,"")
        else:
            hash_C_til[i] = (i,hash_fun(reads_err_ind[i][1],a,w,l))    
    hash_C_til.sort(key=lambda x:x[1])

    cnt = 0
    for i in range(0,len(hash_C_til)-1):
        if(hash_C_til[i][1] == ""):
            continue 
        else:
            if hash_C_til[i][1] == hash_C_til[i+1][1]:
                x = reads_err[hash_C_til[i][0]]
                y = reads_err[hash_C_til[i+1][0]]
                if((ham_dis(bin_sig_arr[hash_C_til[i][0]],bin_sig_arr[hash_C_til[i+1][0]]) <= th_low) or ((ham_dis(bin_sig_arr[hash_C_til[i][0]],bin_sig_arr[hash_C_til[i+1][0]]) <= th_high) and edit_dis(x,y)<=r)):
                    cnt +=  1                   
                    min_temp,max_temp = min_max(rep_find(hash_C_til[i][0],parent),rep_find(hash_C_til[i+1][0],parent))                    
                    C_til[min_temp].extend(C_til[max_temp])
                    C_til[max_temp] = []        
                    parent[max_temp] = min_temp                    

    if lcl_step<=10:
        temp1 = 2
        temp2 = 1
    elif lcl_step>10 and lcl_step<70:
        temp1 = 10
        temp2 = 5
    else:
        temp1 = 40
        temp2 = 20
                    
    if (lcl_step%temp1)==temp2 :
        time_acr_cal_start = time.time()
        acrcy_dict1[lcl_step+1] = calc_acrcy(C_til,C_dict,C_reps,0.6)/card_C
        acrcy_dict2[lcl_step+1] = calc_acrcy(C_til,C_dict,C_reps,0.8)/card_C
        acrcy_dict3[lcl_step+1] = calc_acrcy(C_til,C_dict,C_reps,0.9)/card_C
        acrcy_dict4[lcl_step+1] = calc_acrcy(C_til,C_dict,C_reps,0.95)/card_C
        acrcy_dict5[lcl_step+1] = calc_acrcy(C_til,C_dict,C_reps,0.99)/card_C
        acrcy_dict6[lcl_step+1] = calc_acrcy(C_til,C_dict,C_reps,1)/card_C
        print("Accuracy:",acrcy_dict1[lcl_step+1],acrcy_dict3[lcl_step+1],acrcy_dict5[lcl_step+1],acrcy_dict6[lcl_step+1] )
        time_acr_cal = time.time() - time_acr_cal_start
        time_acrcy_dict[lcl_step+1]= time_acr_cal
    
    time_itr = time.time() - time_start
    time_itr_dict[lcl_step+1]=time_itr
    
    print(time_itr,lcl_step,cnt)
    
#     lcl_step += 1

6.71117091178894 0 238390
Accuracy: 0.2727935054121565 0.02487510407993339 0.0012489592006661116 0.0009367194004995837
80.98542404174805 1 41036
6.701944351196289 2 228855
Accuracy: 0.5698376353039134 0.13728143213988342 0.01582014987510408 0.01186511240632806
63.91663718223572 3 94548
1.5623161792755127 4 1341
Accuracy: 0.5712947543713572 0.13800999167360534 0.01582014987510408 0.01186511240632806
61.45423173904419 5 337
5.3834052085876465 6 164648
Accuracy: 0.8413821815154038 0.3864487926727727 0.07212739383846795 0.050374687760199835
47.18474054336548 7 142039
1.1120245456695557 8 251
Accuracy: 0.8671940049958368 0.43151540383014153 0.09231890091590342 0.0656744379683597
44.31160092353821 9 31390
1.593775749206543 10 25190
1.1107473373413086 11 645
1.940969705581665 12 39583
1.284520149230957 13 11349
1.373311996459961 14 15796
Accuracy: 0.953684429641965 0.6517485428809325 0.22398001665278935 0.16465445462114905
36.50207257270813 15 20717
1.2106432914733887 16 8126
1.25816106796264

In [83]:
with open('time_itr_dict_sample.json', 'w') as fp:
    json.dump(time_itr_dict, fp)
with open('time_acrcy_dict_sample.json', 'w') as fp:
    json.dump(time_acrcy_dict, fp)
with open('acrcy_dict1_sample.json', 'w') as fp:
    json.dump(acrcy_dict1, fp)
with open('acrcy_dict2_sample.json', 'w') as fp:
    json.dump(acrcy_dict2, fp)
with open('acrcy_dict3_sample.json', 'w') as fp:
    json.dump(acrcy_dict3, fp)
with open('acrcy_dict4_sample.json', 'w') as fp:
    json.dump(acrcy_dict4, fp)
with open('acrcy_dict5_sample.json', 'w') as fp:
    json.dump(acrcy_dict5, fp)
with open('acrcy_dict6_sample.json', 'w') as fp:
    json.dump(acrcy_dict6, fp)

# with open('time_itr_dict_12,8,6,300_clstr_dict_block_apprch.json') as f:
#     time_itr_dict = json.load(f)

In [84]:
total_time = sum(time_itr_dict.values())
total_time

751.5847170352936

In [85]:
total_time_acrcy =sum(time_acrcy_dict.values())
total_time_acrcy

541.1849417686462

In [86]:
alg_run_time = total_time-total_time_acrcy 
alg_run_time

210.39977526664734

In [87]:
keys = acrcy_dict1.keys()
values1 = acrcy_dict1.values()
values2 = acrcy_dict2.values()
values3 = acrcy_dict3.values()
values4 = acrcy_dict4.values()
values5 = acrcy_dict5.values()
values6 = acrcy_dict6.values()

df = pd.DataFrame()

df["keys"] = keys
df["0.6"]= values1
df["0.8"]= values2
df["0.9"]= values3
df["0.95"]= values4
df["0.99"]= values5
df["1.0"]= values6

fig = px.line(df, x=df["keys"], y=['0.6', '0.8', '0.9', '0.95', '0.99', '1.0'])
fig.show()

In [88]:
keys = time_itr_dict.keys()
values = time_itr_dict.values()

px.line(x=keys,y=values)

In [90]:
 !git status 

On branch master
Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	deleted:    Clustering-algorithm-single-core-(multiple-values-of-gamma)-blocking-approach.ipynb
	deleted:    Clustering-algorithm-single-core-(multiple-values-of-gamma).ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	../CV.pdf
	.ipynb_checkpoints/
	Clustering-algorithm-Erlich-(q=3-num_blocks=2).ipynb
	Clustering-algorithm-sample.html
	Clustering-algorithm-sample.ipynb
	ErlichDesign.csv
	LuisDesign.txt
	acrcy_dict1_sample.json
	acrcy_dict2_sample.json
	acrcy_dict3_sample.json
	acrcy_dict4_sample.json
	acrcy_dict5_sample.json
	acrcy_dict6_sample.json
	acrcy_dict_20,45,15,0.6,720_clstr_dict-Erlich-cut(q=4).json
	acrcy_dict_20,45,15,0.8,720_clstr_dict-Erlich-cut(q=4).json
	acrcy_dict_20,45,15,0.9,720_clstr_dict-Erlich-cut(q=4).json
	acrcy_dict_20,45,15,0.95,720_clstr

In [91]:
!git add Clustering-algorithm-sample.ipynb

In [65]:
!git add -u && git status && git commit -m "Samp" && git push origin master 